In [1]:
import numpy as np
import pandas as pd

In [73]:
def initialize():
  import numpy as np
  import pandas as pd

  rating = pd.read_csv('rating.csv')
  anime = pd.read_csv('anime.csv')


  genres = set()
  for i in anime['genre']:
      if type(i) != float:
          for j in i.split(', '):
              genres.add(j)
  genres = list(genres)
  anime = anime.drop(['type', 'episodes', 'rating', 'members'], axis=1)
  # add generes list as columns in anime dataframe
  ad = pd.DataFrame(columns=genres)
  anime = pd.concat([anime, ad], axis=1)
  # add 1 if anime has that genre else 0
  for i in range(len(anime)):
      if type(anime['genre'][i]) != float:
          for j in anime['genre'][i].split(', '):
              anime[j][i] = 1
  # drop unnecessary columns
  anime = anime.drop(['genre'], axis=1)
  # fillna
  anime = anime.fillna(0)
  return anime,rating

In [74]:
def u_profile(anime,rating,user_id):



  user_ratings = rating[rating['user_id'] == user_id]
  user_anime = anime[anime['anime_id'].isin(user_ratings['anime_id'])]

  user_data = pd.merge(user_ratings, user_anime, on='anime_id')
  #user_data['ratings']= user_data['rating'].replace(-1,0)
  user_data.replace(-1, 0, inplace=True)

  user_data = user_data.drop(['user_id','anime_id','name'],axis=1)
  ratings= user_data['rating']
  user_data.drop(['rating'],axis=1,inplace=True)


  ratings_arr = ratings.to_numpy()
  user_data_arr = user_data.to_numpy()

  weighted_genres_arr = ratings_arr.reshape(-1, 1) * user_data_arr


  weighted_genres = pd.DataFrame(weighted_genres_arr, columns=user_data.columns)
  user_profile = weighted_genres.sum(axis=0)
  total_sum = user_profile.sum()
  normalized_profile = user_profile / total_sum

  return normalized_profile

In [75]:
def predict(anime,rating,user_id):


  user_ratings = rating[rating['user_id'] == user_id]
  user_anime_ids = user_ratings['anime_id'].tolist()

  unwatched= anime[~anime['anime_id'].isin(user_anime_ids)]
  unwatched_genres = unwatched.drop(['anime_id','name'],axis = 1)

  normalized_profile = u_profile(anime,rating,user_id)

  normalized = normalized_profile.to_numpy()
  unwatched_array = unwatched_genres.to_numpy()

  recommend = np.dot(unwatched_array, normalized.T)
  recommend*=10

  recommend = pd.DataFrame(recommend)

  unwatched = unwatched.reset_index(drop=True)
  recommend = recommend.reset_index(drop=True)


  result = pd.concat([unwatched, recommend], axis=1)
  result = result.rename(columns={0: 'Predicted_rating'})


  top_10 = result.nlargest(10, 'Predicted_rating')
  top = top_10[['name','Predicted_rating']]

  return top





In [78]:
anime,rating = initialize()

print('Enter User ID')
x = int(input())

top = predict(anime,rating,x)
top

<ipython-input-73-8467968cebf9>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anime[j][i] = 1


Enter User ID
10


,name,Predicted_rating
281,Tsubasa: Shunraiki,8.250
10916,Berserk (2017),8.250
196,Fullmetal Alchemist,7.750
1551,Fullmetal Alchemist: The Sacred Star of Milos,7.750
129,Tsubasa: Tokyo Revelations,7.625
147,Berserk,7.625
169,InuYasha: Kanketsu-hen,7.625
369,Log Horizon,7.625
685,InuYasha,7.625
1064,Tsubasa Chronicle 2nd Season,7.625
